<a href="https://colab.research.google.com/github/pratikiiitb2013/NLP_END_P1/blob/main/Session9/END_Session9_Seq2Seq_using_RNN_Encoder_Decoder_Attention(Coached_Conversational_Preference_Elicitation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from requests import request
import json
pd.set_option('display.max_colwidth', -1)

import ast

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext import data
from torchtext.data import Field, BucketIterator

import spacy

import random
import math
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


# Coached Conversational Preference Elicitation

## About the dataset - 

A data set of 502 dialogues with 12,000 annotated statements between a user and a wizard discussing natural language movie preferences. The data were collected using the Oz Assistant method between two paid workers, one of whom acts as an “assistant” and the other as a “user”.

The dialogue corpus is provided in JSON format in the file data.json.

Each conversation has the following fields:
  conversationId: A unique random ID for the conversation. The ID has no meaning.
  utterances: An array of utterances by the workers.

## Downloading and extracting the dataset

In [2]:
response=request(url='https://storage.googleapis.com/dialog-data-corpus/CCPE-M-2019/data.json', method='get')
elevations = response.json()
elevations
s1 = json.dumps(elevations)
dt = json.loads(s1)
df = pd.json_normalize(dt)

In [3]:
print(df.shape)
df.head()

(502, 2)


conversationId                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

## Preprocessing data for model input

#### dataset included different conversation exchanges( can be more than 2 dialogues in a single exchange). So we will be processing dataset such that each consecutive pair of dialogues will be source and target dialogues

In [4]:
df['list_txt'] = df.utterances.apply(lambda x : [ sub['text'] for sub in x]) 
df['pairs'] = df['list_txt'].apply(lambda x : list(zip(x, x[1:])))
df2 = pd.DataFrame(
      np.concatenate(df['pairs'].values), 
      columns = ['dialogue1','dialogue2'])
print(df2.shape)
df2.head()

(11469, 2)


,dialogue1,dialogue2
0,generally speaking what type of movies do you watch,I like thrillers a lot.
1,I like thrillers a lot.,thrillers? for example?
2,thrillers? for example?,Zodiac's one of my favorite movies.
3,Zodiac's one of my favorite movies.,"Zodiac the movie about a serial killer from the '60s or '70s, around there."
4,"Zodiac the movie about a serial killer from the '60s or '70s, around there.","Zodiac? oh wow ok, what do you like about that movie"


### We have divided conversation history into pairs of dialogues(dialogue1 and dialogue2). Now we will input these pairs seq2seq model

In [5]:
SRC = Field(tokenize='spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

TRG = Field(tokenize = 'spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [6]:
fields = [('dialogue1', SRC),('dialogue2',TRG)]

In [7]:
example = [data.Example.fromlist([df2.dialogue1[i],df2.dialogue2[i]], fields) for i in range(df2.shape[0])] 

In [8]:
uttranceDataset = data.Dataset(example, fields)

In [9]:
(train_data, valid_data, test_data) = uttranceDataset.split(split_ratio=[0.70, 0.10, 0.20], random_state=random.seed(SEED))

In [10]:
(len(train_data), len(valid_data), len(test_data))

(8028, 2294, 1147)

In [11]:
print(vars(train_data.examples[1]))
print(vars(train_data.examples[0]))

{'dialogue1': ['what', 'do', 'you', 'like', 'about', 'this', 'movie', '?', '2', 'reasons', 'please'], 'dialogue2': ['i', 'thought', 'the', 'special', 'effects', 'were', 'fantastic', ',', 'and', 'i', 'thought', 'the', 'acting', 'overall', 'was', 'very', 'believable', '.']}
{'dialogue1': ['just', 'not', 'a', 'fan', 'of', 'it', '.'], 'dialogue2': ['it', 'did', "n't", 'really', 'made', 'me', 'feel', 'like', 'wow', ',', 'this', 'is', 'a', 'good', 'movie', '.', 'it', 'just', 'did', "n't", 'make', 'me', 'feel', 'excited', 'about', 'it', '.']}


In [12]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [13]:
print('Size of input vocab : ', len(SRC.vocab))
print('Size of label vocab : ', len(TRG.vocab))
print('Top 10 words appreared repeatedly :', list(SRC.vocab.freqs.most_common(10)))

Size of input vocab :  2326
Size of label vocab :  2350
Top 10 words appreared repeatedly : [('.', 6863), ('i', 5047), (',', 4535), ('like', 4256), ('the', 4185), ('you', 4141), ('it', 4010), ('?', 3268), ('of', 2930), ('a', 2627)]


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [15]:

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), batch_size = BATCH_SIZE, 
                                                            sort_key = lambda x: len(x.dialogue1),
                                                            sort_within_batch=True, device = device)



In [16]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [17]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [18]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [19]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [20]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [21]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2326, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(2350, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=2350, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,843,886 trainable parameters


In [23]:
optimizer = optim.Adam(model.parameters())

In [24]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [25]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.dialogue1
        trg = batch.dialogue2
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [26]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.dialogue1
            trg = batch.dialogue2

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [27]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [28]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')



Epoch: 01 | Time: 0m 45s
	Train Loss: 5.469 | Train PPL: 237.146
	 Val. Loss: 4.999 |  Val. PPL: 148.338
Epoch: 02 | Time: 0m 47s
	Train Loss: 4.807 | Train PPL: 122.355
	 Val. Loss: 4.881 |  Val. PPL: 131.730
Epoch: 03 | Time: 0m 48s
	Train Loss: 4.525 | Train PPL:  92.299
	 Val. Loss: 4.884 |  Val. PPL: 132.117
Epoch: 04 | Time: 0m 48s
	Train Loss: 4.375 | Train PPL:  79.475
	 Val. Loss: 4.878 |  Val. PPL: 131.343
Epoch: 05 | Time: 0m 48s
	Train Loss: 4.262 | Train PPL:  70.967
	 Val. Loss: 4.915 |  Val. PPL: 136.254
Epoch: 06 | Time: 0m 49s
	Train Loss: 4.181 | Train PPL:  65.459
	 Val. Loss: 4.999 |  Val. PPL: 148.196
Epoch: 07 | Time: 0m 48s
	Train Loss: 4.150 | Train PPL:  63.406
	 Val. Loss: 4.822 |  Val. PPL: 124.179
Epoch: 08 | Time: 0m 48s
	Train Loss: 4.099 | Train PPL:  60.283
	 Val. Loss: 4.849 |  Val. PPL: 127.666
Epoch: 09 | Time: 0m 48s
	Train Loss: 3.978 | Train PPL:  53.403
	 Val. Loss: 4.952 |  Val. PPL: 141.455
Epoch: 10 | Time: 0m 49s
	Train Loss: 3.947 | Train PPL

In [29]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.808 | Test PPL: 122.538 |
